# Preparation

In [6]:
'''Libraries'''
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import shutil
import numpy as np
import cv2
from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import densenet, efficientnet, resnet_v2
import tensorflow as tf

from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from keras import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
'''HYPERPARAM'''
FACTOR = 2 # reduce train and val data size by FACTOR due to memory limit # 2
IMG_DIM = [64, 64, 3] # reduce img size from [218, 178, 3] due to memory limit
BATCH_SZ = 8
N_CLASSES = 40
EPOCHS = 2 # 20

In [8]:
'''Train, val, test indices'''
TRAIN_LAST = 162770
VAL_LAST = 182637
TEST_LAST = 202599
train_idx = [i for i in range(TRAIN_LAST)]
val_idx = [i for i in range(TRAIN_LAST, VAL_LAST)]
test_idx = [i for i in range(VAL_LAST, TEST_LAST)]

# Preprocessing

In [ ]:
'''Path to dataset files'''
base_dir = 'celeba-dataset/'
x_path = "%s%s" % (base_dir,'img_align_celeba/img_align_celeba/')
y_path = "%s%s" % (base_dir,'list_attr_celeba.csv')

In [ ]:
'''Labels'''
y_df = pd.read_csv(y_path)
new_y_df = y_df.copy()
new_y_df.replace(to_replace=-1, value=0, inplace=True)

In [ ]:
'''Process train labels and iamges'''
y_train_df = new_y_df.drop(val_idx+test_idx, axis=0)
y_train_df = y_train_df.sample(int(len(train_idx)/FACTOR), random_state=42)
y_train_df.set_index('image_id', inplace=True)
y_train = y_train_df.to_numpy(dtype='float16')

x_train = np.zeros(tuple([int(len(train_idx)/FACTOR)]+ IMG_DIM), dtype='float16')
for i, x_name in enumerate(tqdm(np.asarray(y_train_df.index))):
    img = load_img(x_path + x_name, target_size=tuple(IMG_DIM[:-1]))
    x = img_to_array(img, dtype='float16')/255.0
    x_train[i] = x

In [ ]:
'''Process val labels and images'''
y_val_df = new_y_df.drop(train_idx+test_idx, axis=0)
y_val_df = y_val_df.sample(int(len(val_idx)/FACTOR))
y_val_df.set_index('image_id', inplace=True)
y_val = y_val_df.to_numpy(dtype='float16')

x_val = np.zeros(tuple([int(len(val_idx)/FACTOR)] + IMG_DIM), dtype='float16')
for i, x_name in enumerate(tqdm(np.asarray(y_val_df.index))):
    img = load_img(x_path + x_name, target_size=tuple(IMG_DIM[:-1]))
    x = img_to_array(img, dtype='float16')/255.0
    x_val[i] = x

In [ ]:
'''Process test labels and images'''
y_test_df = new_y_df.drop(train_idx+val_idx, axis=0)
y_test_df.set_index('image_id', inplace=True)
y_test = y_test_df.to_numpy(dtype='float16')
x_test = np.zeros(tuple([int(len(test_idx))] + IMG_DIM), dtype='float16')
for i, x_name in enumerate(y_test_df.index):
    img = load_img(x_path + x_name, target_size=tuple(IMG_DIM[:-1]))
    x = img_to_array(img, dtype='float16')/255.0
    x_test[i] = x

In [ ]:
'''Save processed data'''
np.save('./x_train', x_train)
np.save('./x_val', x_val)
np.save('./x_test', x_test)
np.save('./y_train', y_train)
np.save('./y_val', y_val)
np.save('./y_test', y_test)

# Training

In [ ]:
base_dir = 'celeba-final/'
x_train_path = "%s%s" % (base_dir,'x_train.npy')
x_val_path = "%s%s" % (base_dir,'x_val.npy')
y_train_path = "%s%s" % (base_dir,'y_train.npy')
y_val_path = "%s%s" % (base_dir,'y_val.npy')
x_train = np.load(x_train_path)
x_val = np.load(x_val_path)
y_train = np.load(y_train_path)
y_val = np.load(y_val_path)

In [ ]:
# preprocess_input = densenet.preprocess_input
# preprocess_input = resnet_v2.preprocess_input
preprocess_input = efficientnet.preprocess_input

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# imagedatagenerator param from deep learning textbook
# train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
#                                    rotation_range=40,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest')
train_datagen.fit(x_train)
train_generator = train_datagen.flow(x_train, y_train, batch_size=BATCH_SZ)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen.fit(x_val)
val_generator = val_datagen.flow(x_val, y_val, batch_size=BATCH_SZ)

In [14]:

def create_model():
#     base_model = densenet.DenseNet201(input_shape=tuple(IMG_DIM), include_top=False)
#     base_model = resnet_v2.ResNet152V2(input_shape=tuple(IMG_DIM), include_top=False)
    base_model = efficientnet.EfficientNetB7(input_shape=tuple(IMG_DIM), include_top=False) # comment out if previous error!
    # base_model = tf.keras.applications.EfficientNetB7(input_shape=tuple(IMG_DIM), include_top=False) # TensorFlow version
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
#     x = Dense(512, activation="relu")(x)
    outputs = Dense(N_CLASSES, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

In [ ]:
model = create_model()
model.compile(optimizer=SGD(lr=0.1, decay=0.0001, momentum=0.9, nesterov=True), loss='binary_crossentropy', metrics=[metrics.BinaryAccuracy()]) # change this in other notebook

In [ ]:
# early = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True, verbose=1)

In [ ]:
# generator workers, use_multiprocessing
hist = model.fit_generator(train_generator,
                           steps_per_epoch=int(len(train_idx)/FACTOR/BATCH_SZ),
                           validation_data=val_generator,
                           validation_steps=int(len(val_idx)/FACTOR/BATCH_SZ),
                           epochs=EPOCHS,
                           workers=2)

In [ ]:
'''Fine tuning'''
## Total params: 64,200,127

# layer_name = 'block7d_expand_conv' # 7,945,160
# layer_name = 'block7c_expand_conv' # 14,000,000
# set_trainable = False
# for layer in model.layers:
#     if layer.name == layer_name:
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False
# model.compile(optimizer=SGD(lr=0.0001, decay=0.0001, momentum=0.9, nesterov=True), loss='binary_crossentropy', metrics=[metrics.BinaryAccuracy()])
# early = EarlyStopping(monitor='val_loss', patience=1, mode='min', restore_best_weights=True, verbose=1)
# hist = model.fit_generator(train_generator,
#                            steps_per_epoch=int(len(train_idx)/FACTOR/BATCH_SZ),
#                            validation_data=val_generator,
#                            validation_steps=int(len(val_idx)/FACTOR/BATCH_SZ),
#                            epochs=int(EPOCHS/2),
#                            workers=2)

In [ ]:
'''Save model'''
model.save_weights("./final_weights.h5") # i think i can just save weights actually would be easier to just save the model - larger size tho?

# Evaluation

In [10]:
base_dir = 'celeba-final/'
x_test_path = "%s%s" % (base_dir,'x_test.npy')
x_test = np.load(x_test_path)
y_test_path = "%s%s" % (base_dir,'y_test.npy')
y_test = np.load(y_test_path)

In [ ]:
model = create_model()
model.compile(optimizer=SGD(lr=0.1, decay=0.0001, momentum=0.9, nesterov=True), loss='binary_crossentropy', metrics=[metrics.BinaryAccuracy()]) # change this in other notebook
path_to_weights = './final_weights.h5'
model.load_weights(path_to_weights)

In [ ]:
model.evaluate(x_test, y_test, batch_size=BATCH_SZ)[1]